### List of all products 

In [33]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import json
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
import requests

BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
ENDPOINT='/products'



url = BASE_URL + ENDPOINT

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}
productlist = []
listlength = 0
i = 0
while True:
    query = {
        'page[number]': i,
        'page[size]': 200  # maximum
    }
    r = requests.get(url, params=query, headers=headers)
    if r.status_code == 200:
        print('Page ' + str(i) + ' loaded successfully.')

        for product in r.json()['data']:
            try:
                productlist.append(product['id'])
            except:
                pass

    if listlength == len(productlist):
        break
    #print('Product count: ' + str(len(productlist)))
    listlength = len(productlist)
    i += 1
    
'''
url = BASE_URL + ENDPOINT

query = {
  'page[number]': 1,
  'page[size]': 200
}

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}

r = requests.get(url, params=query, headers=headers)
print("Status: " + str(r.status_code))
if r.status_code == 200:
    print('Page loaded successfully.')
    print('Generated in ' + str(r.json().get('meta', {}).get('generated_in', -1)) + ' milliseconds.')
    print('Next page\'s URL is: ' + r.json()['links'].get('next', 'UNKNOWN'))
    print('Barcodes on this page:')
    for product in r.json()['data']:
        productlist = []
        for x in r.json()['data']:
            productlist.append(x['id'])
print(productlist)
'''
print(productlist)

Page 0 loaded successfully.
Page 1 loaded successfully.
Page 2 loaded successfully.
Page 3 loaded successfully.
Page 4 loaded successfully.
Page 5 loaded successfully.
Page 6 loaded successfully.
Page 7 loaded successfully.
Page 8 loaded successfully.
Page 9 loaded successfully.
Page 10 loaded successfully.
Page 11 loaded successfully.
Page 12 loaded successfully.
Page 13 loaded successfully.
Page 14 loaded successfully.
Page 15 loaded successfully.
Page 16 loaded successfully.
Page 17 loaded successfully.
Page 18 loaded successfully.
Page 19 loaded successfully.
Page 20 loaded successfully.
Page 21 loaded successfully.
Page 22 loaded successfully.
Page 23 loaded successfully.
Page 24 loaded successfully.
Page 25 loaded successfully.
Page 26 loaded successfully.
Page 27 loaded successfully.
Page 28 loaded successfully.
Page 29 loaded successfully.
Page 30 loaded successfully.
Page 31 loaded successfully.
Page 32 loaded successfully.
Page 33 loaded successfully.
Page 34 loaded successfu

### Getting list of useful characteristics from id

In [ ]:
%%time
BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
dInfo = {}
dIngredients = {}
dNutrients = {}

for PRODUCT_ID in productlist:
    ENDPOINT='/products/{}'.format(PRODUCT_ID)

    url = BASE_URL + ENDPOINT

    headers = {
      'Authorization': 'Token token=' + KEY,
      'Accept': 'application/json',
      'Content-Type': 'application/vnd.api+json',
      'Accept-Encoding': 'gzip,deflate'
    }

    r = requests.get(url, headers=headers)
    #print ('Status: ' + str(r.status_code))
    item = {}
    item_ing = {}
    item_nut = {}
    data = r.json()['data']
    if r.status_code == 200:
        try:
            item['name'] = item_ing['name'] = item_nut['name'] = data['display_name_translations'].popitem()
            item['id'] = item_ing['id'] = item_nut['id'] = data['id']
            #item['barcode'] = item_ing['barcode'] = item_nut['barcode'] = data['barcode']
            
            item['quantity'] = data['quantity']
            item['unit'] = data['unit']
            item['portion'] = data['portion_quantity']
            item['portion unit'] = data['portion_unit']
            item['alcohol'] = data['alcohol_by_volume']
            
            item_ing['ingredients'] = data['ingredients_translations'].popitem()
            
            nuts = {}
            for nut in data['nutrients'].keys():
                nuts[nut] = {'per_day':data['nutrients'][nut]['per_day'],
                             'per_hundred':data['nutrients'][nut]['per_hundred'],
                             'per_portion':data['nutrients'][nut]['per_portion']}
            item_nut['nutrients'] = nuts
        except:
            pass
    dInfo[data['barcode']] = item    
    dIngredients[data['barcode']] = item_ing
    dNutrients[data['barcode']] = item_nut

### Translate ingredients

In [21]:
%%time
from watson_developer_cloud import LanguageTranslatorV2 as LanguageTranslator

language_translator = LanguageTranslator(username='a69f77dc-9248-4069-85b5-1ac1ef6bfd7d',password='UQznP8xcKtuq')

count = 0
remove = []
for barcode, tmp in dIngredients.items():
    try:
        ingredients_ = ' '.join(tmp['ingredients'][1].splitlines())
        language_json = language_translator.identify(ingredients_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dIngredients[barcode]['ingredients'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dIngredients[barcode]['ingredients'] = "en_kept, " + ingredients_
    except:
        remove.append(barcode)
    count += 1
    if count % 50 == 0:
        print(%s done!%count)

# remove the ones (from everywhere) which couldn't be translated
for barcode in remove:
    del dInfo[barcode]
    del dIngredients[barcode]
    del dNutrients[barcode]

sv Sesamöl
de Aprikosen (Ungarn, Turkei, Bulgarien), Zucker (Schweiz, Deutschland), Zitronensaftkonzentrat, Geliermittel(E 440i).  Alle landwirtschaftlichen Zutaten stammen aus biologischem Anbau.  Hegestellt aus 54g Früchten je 100g Fertigptodukt
de Nudeln 91,8% [Weizenmehl, Palmöl, Kochsalz, Mehlbehandlungsmittel (E500, E451), Stabilisator E501, Verdickungsmittel E412, Antioxidationsmittel E306], Streuwüze 8,2% [Kochsalz, Zucker, Geschmacksverstärker (E621, E635), Knoblauch, Maltodextrin, Gewürze, Aromen, Porree, Säureregulator: Citronensäure, Hühneraroma (0,5% in Streuwürze; Geschmacksverstärker: E627, E631; Weizen, Soja), Trennmittel E551].
de Linsenmehl 71%, Sonnenblumenöl, Meersalz, Backtriebmittel: E529, Reismehl.
de Wasser, Avocado (14%), Tomaten (10%), Zwiebeln (7%), Weizenmehl, Frischkäse, Rahm, modifizierte Maisstärke (enthält Sulphite), Rapsöl, Kartoffelflocken (enthält Sulphite), Knoblauchpüree (enthält Sulphite), Molkenextrakt (Milch), Molkenpulver (Milch), Speisesalz, Ge

In [28]:
# dump dictionaries to json files (load it in with `json.load(fName)`)

with open("dInfo.json", "w") as f_:
    json.dump(dInfo, f_)
with open("dIngredients.json", "w") as f_:
    json.dump(dIngredients, f_)
with open("dNutrients.json", "w") as f_:
    json.dump(dNutrients, f_)